In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score

# Prepare the Data 

Read the data and drop unneeded columns.

In [3]:
product_df = pd.read_json("data/products_table_for_linear_regression.json")


In [4]:
product_df.drop(["id","product_name","category", "product_description","location","page_id","create_time", "main_category"],axis=1, inplace=True)

In [5]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7009 entries, 1 to 8090
Data columns (total 15 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   price                                        7009 non-null   float64
 1   longitude                                    7009 non-null   float64
 2   latitude                                     7009 non-null   float64
 3   main_category_baby_kids_stuff                7009 non-null   int64  
 4   main_category_clothes_footwear_accessories   7009 non-null   int64  
 5   main_category_computers_software             7009 non-null   int64  
 6   main_category_diy_tools_materials            7009 non-null   int64  
 7   main_category_health_beauty                  7009 non-null   int64  
 8   main_category_home_garden                    7009 non-null   int64  
 9   main_category_music_films_books_games        7009 non-null   int64  
 10  


# Prepare Data

In [6]:
X = product_df.drop(["price"], axis=1)
y = product_df["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [7]:
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)


# Linear Regression

In [8]:
linear_regression_model = LinearRegression().fit(X_train_std, y_train)
y_prediction = linear_regression_model.predict(X_test_std)
r2_score(y_test, y_prediction)

0.04239606569022869

# Implement a Grid Search for Ridge model

In [29]:
# Create model
ridge = Ridge(alpha=1).fit(X_train, y_train)

# Select hyperparameters to search over
alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

param_grid = dict(alpha=alpha)

# Perform the grid search
grid = GridSearchCV(estimator=ridge, param_grid=param_grid, scoring='r2', verbose=1, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Fitting 5 folds for each of 7 candidates, totalling 35 fits
Best Score:  0.08755583314162077
Best Params:  {'alpha': 10}


# Implement a Grid Search for Lasso Model

In [30]:
# Create model
lasso = Lasso(alpha=1).fit(X_train, y_train)

# Select hyperparameters
alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

param_grid = dict(alpha=alpha)

# Perform grid search
grid = GridSearchCV(estimator=lasso, param_grid=param_grid, scoring='r2', verbose=1, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Fitting 5 folds for each of 7 candidates, totalling 35 fits
Best Score:  0.08751546567611986
Best Params:  {'alpha': 0.1}


# Implemenet a Grid Search for Elastic Model

In [31]:
# Create model
elastic_net = ElasticNet(alpha=1, l1_ratio=0.5).fit(X_train, y_train)

# Select hyperparameters
alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
l1_ratio = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

param_grid = dict(alpha=alpha, l1_ratio=l1_ratio)

# Perform grid search
grid = GridSearchCV(estimator=elastic_net, param_grid=param_grid, scoring='r2', verbose=1, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Fitting 5 folds for each of 77 candidates, totalling 385 fits


/home/amy/miniconda3/envs/rec_ranking/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.123e+07, tolerance: 1.349e+04 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/home/amy/miniconda3/envs/rec_ranking/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.945e+07, tolerance: 1.312e+04 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using

Best Score:  0.08755209813157169
Best Params:  {'alpha': 0.01, 'l1_ratio': 0.7}


/home/amy/miniconda3/envs/rec_ranking/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.766e+07, tolerance: 1.355e+04 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/home/amy/miniconda3/envs/rec_ranking/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.770e+07, tolerance: 1.355e+04 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using